In [ ]:
import pandas as pd
from tabulate import tabulate
import os

In [4]:
files = os.listdir("./DATA/")
files.sort(reverse=True)
file = files[0] #me quedo con el último
csv_file = f'./DATA/{file}'  # Replace with the path to your CSV file
df = pd.read_csv(csv_file)

In [6]:
df.head()

,estado,tipooferta,jornada,ige,miercoles,martes,acargodireccion,cuilautor,supl_hasta,turno,...,reemp_motivo,descdistrito,jueves,nivelmodalidad,viernes,descripcionarea,vuelta,descripcioncargo,ult_movimiento,_version_
0,Publicada,DESIGNACIONES DOCENTES,JS,3109065,NaN,NaN,No,20147633158,9999-12-30T12:53:53.520Z,M,...,NaN,BERAZATEGUI,1050 A 1250,E,NaN,INTRODUCCIÓN A LA COMUNICACIÓN (IAC),NaN,INTRODUCCIÓN A LA COMUNICACIÓN (IAC),2025-03-10T09:53:53.477Z,1826303837397319686
1,Publicada,DESIGNACIONES DOCENTES,JS,3115579,7:30 A 12:00,7:30 A 12:00,No,20275425894,9999-12-30T11:18:07.177Z,M,...,NaN,BERAZATEGUI,7:30 A 12:00,A,7:30 A 12:00,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-03-11T08:18:07.223Z,1826303839240716290
2,Publicada,DESIGNACIONES DOCENTES,JS,3116770,NaN,NaN,No,20147633158,2026-02-28T00:00:00Z,T,...,Licencia no medica por ARTICULO 114(ART114NOme...,BERAZATEGUI,NaN,E,17:20 A 18:20,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:13:01.070Z,1826303839711526918
3,Publicada,DESIGNACIONES DOCENTES,JS,3116245,9.30 A 10.30,NaN,No,20147633158,9999-12-30T12:30:21.887Z,M,...,NaN,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:30:23.140Z,1826303839739838466
4,Publicada,DESIGNACIONES DOCENTES,JS,3116363,NaN,NaN,No,20275425894,2026-02-28T00:00:00Z,M,...,Licencia no medica por ARTICULO 115(ART115),BERAZATEGUI,7:40 A 9:40,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:39:28.980Z,1826303839739838467


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   estado                 10 non-null     object 
 1   tipooferta             10 non-null     object 
 2   jornada                10 non-null     object 
 3   ige                    10 non-null     int64  
 4   miercoles              5 non-null      object 
 5   martes                 5 non-null      object 
 6   acargodireccion        10 non-null     object 
 7   cuilautor              10 non-null     int64  
 8   supl_hasta             10 non-null     object 
 9   turno                  10 non-null     object 
 10  idoferta               10 non-null     int64  
 11  domiciliodesempeno     10 non-null     object 
 12  cuilganador            0 non-null      float64
 13  sabado                 0 non-null      float64
 14  listadoorigenganador   0 non-null      float64
 15  id       

In [ ]:
df

In [51]:
ofertas_por_distrito = df.groupby(['descdistrito','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_distrito.columns = ['Distrito','Cargo','Cantidad']
ofertas_por_distrito = tabulate(
    ofertas_por_distrito, 
    headers=ofertas_por_distrito.columns, 
    tablefmt="HTML",
    showindex=False,
)
print(ofertas_por_distrito)


Distrito     Cargo                                   Cantidad
-----------  ------------------------------------  ----------
BERAZATEGUI  CONSTRUCCION DE CIUDADANIA (CCD)               6
BERAZATEGUI  FONOAUDIOLOGO (/FO)                            1
BERAZATEGUI  INTRODUCCIÓN A LA COMUNICACIÓN (IAC)           1
BERAZATEGUI  MAESTRO DE CICLO (/MC)                         1
BERAZATEGUI  PRECEPTOR (/PR)                                1


In [50]:
ofertas_por_escuela = df.groupby(['escuela','domiciliodesempeno','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_escuela.columns = ['Escuela','Dirección','Cargo','Cantidad']
ofertas_por_escuela = tabulate(
    ofertas_por_escuela, 
    headers=ofertas_por_escuela.columns, 
    tablefmt="HTML",
    showindex=False,
)
print(ofertas_por_escuela)

Escuela     Dirección              Cargo                                   Cantidad
----------  ---------------------  ------------------------------------  ----------
0119DF0401  150                    MAESTRO DE CICLO (/MC)                         1
0119JI0911  75 ESQ. 125            FONOAUDIOLOGO (/FO)                            1
0119MS0018  27 E/ 106 Y 107        CONSTRUCCION DE CIUDADANIA (CCD)               1
0119MS0022  129                    INTRODUCCIÓN A LA COMUNICACIÓN (IAC)           1
0119MS0033  75                     CONSTRUCCION DE CIUDADANIA (CCD)               1
0119MS0040  1                      CONSTRUCCION DE CIUDADANIA (CCD)               1
0119MS0042  CALLE 27 E/ 159 Y 160  CONSTRUCCION DE CIUDADANIA (CCD)               1
0119MT0003  15                     CONSTRUCCION DE CIUDADANIA (CCD)               1
0119MT0005  368 ESQ 316            CONSTRUCCION DE CIUDADANIA (CCD)               1
0119MT0006  CALLE 150 E/46 Y 47    PRECEPTOR (/PR)                          

In [21]:
cantidad_ofertas = df["ige"].nunique()
cantidad_ofertas

10